### Spark Session

In [0]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName("HelloWorldStreaming")
        .getOrCreate()
)

### Read Stream

In [0]:
df_news = (
    spark.readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", "localhost:9092,localhost:9095,localhost:9093")
        .option("subscribe", "topic_news")
        .load()
)

df_news

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

### __Console

In [0]:
query = (
    df_news.writeStream
        .format("console")
        .outputMode("append")
        .start()
)

### Schema StructType

In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, NullType

source_schema = StructType([
    StructField("id", StringType()),
    StructField("name", StringType())
])
    
message_schema = StructType(
    [
        StructField("source", source_schema),
        StructField("author", StringType()),
        StructField("title", StringType()),
        StructField("description", StringType()),
        StructField("url", StringType()),
        StructField("urlToImage", StringType()),
        StructField("publishedAt", StringType()),
        StructField("content", StringType()),
    ]
)

df_news_data = df_news.withColumn("value", from_json(col("value").cast("STRING"), message_schema))

df_news_data = df_news_data.selectExpr("value.*")

df_news_data.writeStream.format("console").start()


### __Load


In [0]:
df_news_data.writeStream.format("parquet").option("checkpointLocation", "dbfs:/tmp/checkpoint").option("path", "dbfs:/tmp/output/raw/").start()

### __Extract

In [0]:
spark.read.parquet("dbfs:/tmp/output/raw/").display()

source,author,title,description,url,urlToImage,publishedAt,content
"List(null, Tecmundo.com.br)",Jorge Marin,Google DeepMind: nova ferramenta de IA prevê doenças genéticas,Um novo programa de inteligência artificial desenvolvido por cientistas do Google DeepMind é capaz de prever se milhões de mutações genéticas presentes em nossa sequência de DNA que são inofensivas ou propensas a causar doenças no futuro. Batizada como AlphaM…,https://www.tecmundo.com.br/ciencia/271822-google-deepmind-nova-ferramenta-ia-preve-doencas-geneticas.htm,https://tm.ibxk.com.br/2023/09/20/20120248396161.jpg,2023-09-20T15:15:00Z,Um novo programa de inteligência artificial desenvolvido por cientistas do Google DeepMind é capaz de prever se milhões de mutações genéticas presentes em nossa sequência de DNA que são inofensivas o… [+2940 chars]
"List(null, Olhardigital.com.br)",Alessandro Di Lorenzo,Estudo descobre novas características genéticas do Alzheimer,"A descoberta, segundo os pesquisadores, pode contribuir para o desenvolvimento futuro de exames de sangue para a doença de Alzheimer O post Estudo descobre novas características genéticas do Alzheimer apareceu primeiro em Olhar Digital.",https://olhardigital.com.br/2023/09/07/medicina-e-saude/estudo-descobre-novas-caracteristicas-geneticas-do-alzheimer/,https://img.olhardigital.com.br/wp-content/uploads/2023/05/Alzheimer.jpg,2023-09-07T10:01:00Z,"Pesquisadores da Universidade de Pittsburgh, nos Estados Unidos, analisaram milhares de genomas humanos para encontrar novas variaÃ§Ãµes genÃ©ticas responsÃ¡veis por controlar os nÃ­veis de molÃ©cula… [+2542 chars]"
"List(null, Sapo.pt)",Ana Sofia Neto,Cérebro artificial ajuda a perceber as origens genéticas do autismo,Uma nova técnica científica está a revelar as origens genéticas da perturbação do espetro do autismo (ou PEA) escondidas no nosso cérebro. O cérebro é curiosamente complexo e há muito sobre ele que ainda...,https://pplware.sapo.pt/ciencia/cerebro-artificial-ajuda-a-perceber-as-origens-geneticas-do-autismo/,https://pplware.sapo.pt/wp-content/uploads/2023/09/organoide_cerebral00.jpg,2023-09-24T13:00:25Z,Uma nova técnica científica está a revelar as origens genéticas da perturbação do espetro do autismo (ou PEA) escondidas no nosso cérebro. O cérebro é curiosamente complexo e há muito sobre ele que … [+3282 chars]
"List(null, Tecmundo.com.br)",Jorge Marin,Miniórgãos descobertos em mamíferos podem gerar respostas autoimunes,"Pesquisadores do Instituto Federal Suíço de Tecnologia de Zurique descobriram recentemente um tipo de organela que havia permanecido não caracterizada até agora nas células dos mamíferos. O “miniórgão”, batizado como exclusoma, é feito de anéis de DNA chamado…",https://www.tecmundo.com.br/ciencia/272244-miniorgaos-descobertos-mamiferos-gerar-respostas-autoimunes.htm,https://tm.ibxk.com.br/2023/10/02/02154244691267.jpg,2023-10-02T20:30:04Z,Pesquisadores do Instituto Federal Suíço de Tecnologia de Zurique descobriram recentemente um tipo de organela que havia permanecido não caracterizada até agora nas células dos mamíferos. O miniórgão… [+3134 chars]
"List(null, Olhardigital.com.br)",Vitoria Lopes Gomez,"Vacinas erradicaram doenças que, antes, matavam mais que guerras","Doenças como a poliomielite e a varíola, que mataram milhões no século passado, desapareceram graças às vacinas O post Vacinas erradicaram doenças que, antes, matavam mais que guerras apareceu primeiro em Olhar Digital.",https://olhardigital.com.br/2023/09/08/medicina-e-saude/vacinas-erradicaram-doencas-que-antes-matavam-mais-que-guerras/,https://img.olhardigital.com.br/wp-content/uploads/2023/07/vacina-e1692890816557.jpg,2023-09-08T20:30:20Z,"NÃ£o Ã© incomum ouvir falar sobre a gravidade de doenÃ§as que hoje sequer nos lembramos. Ã o caso, por exemplo, da varÃ­ola e da poliomielite. Ambas mataram milhÃµes de pessoas no sÃ©culo passado, m… [+2308 chars]"
"List(null, Olhardigital.com.br)",Alessandro Di Lorenzo,IA brasileira é capaz de prever doenças com anteced